# Import những modules cần cho ĐỒ ÁN 1
<h3>Modules list: </h3> <ol>
<p><b>json</b>: để lưu dữ liệu từ API <br>
<b>requests</b>: dùng để gửi request kéo dữ liệu về <br>
<b>selenium</b>: dùng để chạy các file JS của một trang web <br>
<b>bs4</b>: để lấy các tag trong HTML của một trang web <br>
<b>time</b>: dùng để cho chương trình ngủ 1 khoảng thời gian trước khi tiếp tục <br>
<b>pandas</b>: dùng để ghi file csv <br>
</p>
</ol>

In [11]:
import requests
import json
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import pandas as pd


### Khởi tạo hàm wait 
<p> Hàm wait có nghĩa đợi cho đến khi lấy được <b> Class name </b> ở trong một tag của HTML </p> 

In [2]:
# wait
def wait(driver, selector_name):
    while (True):
        try:
            myElem = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, selector_name)))
            if myElem is not None:
                break
        except TimeoutException:
            break

### Khởi tạo hàm scroll
<p> Hàm scroll có nghĩa cuộn thanh tác vụ cho đến <b> cuối <i>đường link</i> của trang web đó </b> </p>

In [3]:
# scroll
def scroll(driver):
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(2)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

### Khởi tạo các giá trị 
<p> Đặt đường link chính của trang web cần lấy thông tin <a href="https://soundcloud.com/discover"> soundcloud/discover </a> </p>
<p> Sau đó tạo một mảng dữ liệu 1 chiều để lưu trữ <b> những đường link của người dùng </b> </p>

In [4]:
main_url = "https://soundcloud.com/discover"
set_link = []

### Khởi tạo hàm printProgressBar 
<p> Để tạo thuận tiện cho việc qua sát tiến trình khi chạy chương trình lấy dữ liệu như <b> thông tin người dùng, thông tin của track, thông tin của playlist "%"</b> </p>

In [5]:
import sys

def printProgressBar(i,max,postText):
    n_bar =10 #size of progress bar
    j= i/max
    sys.stdout.write('\r')
    sys.stdout.write(f"[{'=' * int(n_bar * j):{n_bar}s}] {int(100 * j)}%  {postText}")
    sys.stdout.flush()

### Khởi tạo hàm drive_to_url_page
<p> Hàm <b> drive_to_url_page </b> giúp cho <b><i>driver</i></b> của selenium ( ở đây chúng ta sử dụng là <b>Chrome()</b> ) dịch chuyển đến đường bất kì <b><i>đường link </i></b></p>
<p> Sau đó Hàm <b> Scroll </b> sẽ giúp chúng ta lấy nhiều dữ liệu hơn trong HTML </p>

In [6]:
# Drive the chrome to url link and return soup with html parser
def drive_to_url_page(driver, url):
    driver.get(url)
    scroll(driver)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    return soup

# Khởi động Chrome của selenium
<h3> Mục đích để lấy các dữ liệu <i>href</i> ở đây là <b>đường link đẫn đến trang web</b> của người dùng</h3>
<ol>
<li>Tập <i> set_link </i> được cập nhập liên tục  </li>
<li>Sau khi lấy hết <i>href</i> có trong trang chủ chính của <b> SoundCloud</b><ol>
<li> Tiếp đến truy cập vào trang web của người dùng từ những đường link của tập <b>set_link</b> </li>
<li> Chạy đến thông tin những người dùng đang <b>following</b> để cập nhập thêm vào tập <b>set_link</b></li>

</ol>
</ol>

In [7]:
#Driver to soundcloud website
driver = webdriver.Chrome()
soup = drive_to_url_page(driver, main_url)
links = soup.find_all("a", {"class": "sc-link-secondary sc-link-light playableTile__usernameHeading audibleTile__usernameHeading sc-truncate sc-text-h4"})
for i, item in enumerate(links):
    user_url = "https://soundcloud.com" + item.get("href")
    if user_url not in set_link:
        set_link.append(user_url)
    if len(set_link) > 1000:
            break
    
    printProgressBar(i, len(links), "'%'")
#Get the all person in set_link with his/her following information
for i in range(0, len(set_link) - 1):
    if len(set_link) > 1000:
            break
    following_url = set_link[i] + "/following"
    soup = drive_to_url_page(driver, following_url)
    link_following_user = soup.find_all("a", {"class":"userBadgeListItem__heading sc-type-small sc-text-h4 sc-link-dark sc-link-primary sc-truncate"})
    for item in link_following_user:
        if len(set_link) > 1000:
            break
        user_url = "https://soundcloud.com" + item.get("href")
        if user_url not in set_link:
            set_link.append(user_url)
driver.close()

[========= ] 96%  '%'

<h5> Kiểm tra tập dữ liệu xem lấy được bao nhiêu đường link của người dùng</h5>

In [8]:
len(set_link)

1001

### Khởi tạo hàm count_playlist_in_one_person
<p> Mục đích để đếm số lượng playlist của người dùng</p>

In [9]:
#Count all playlist in user_url/sets (count by <li> tag)
def count_playlist_in_one_person(driver, url_of_user):
    playlist_url = str(url_of_user) + '/sets'
    soup = drive_to_url_page(driver, playlist_url)
    playlist_count = soup.find_all("li", {"class":"soundList__item"})
    return len(playlist_count)

# Dự liệu cần từ HTML | Bắt đầu chạy chương trình

## 1. Thông tin từ người dùng

<h3>Khởi tạo hàm get_all_user_data</h3> <br><p> Lấy những dữ liệu cần thiết về bằng sử dụng modules BeautifulSoup</p>
<p> Thông tin lấy được từ <b><i>trang web của người dùng</i></b> </p>
<ol>
<li> ID của người dùng </li>
<li> Tên của người dùng </li>
<li> URL của người dùng </li>
<li> Số lượng followers của người dùng </li>
<li> Số lượng following của người dùng </li>    
<li> Số lượng lượt like tracks của người dùng </li>    
<li> Số lượng Tracks của người dùng </li>        
<li> Số lượng Playlists của người dùng </li>       
</ol>


In [10]:
def get_all_user_data(driver, set_link):
    driver = webdriver.Chrome()    
    #Get user id in SoundCloud/Discover by get meta tag in HEAD html
    set_data_user = []
    for i, link_user in enumerate(set_link):
        one_data_user = []
        driver.get(link_user)
        wait(driver, 'infoStats__value sc-font-light')
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        
        #Get USER INFO
        id_user = soup.find("meta", property="twitter:app:url:googleplay")
        name_user = soup.find("meta", property="og:title")
        url_user = soup.find("meta", property="og:url")
        info_start = soup.find_all("div", {"class":"infoStats__value sc-font-light"})
        
        likeTrack_count = soup.find_all("span", {"class":"sidebarHeader__actualTitle sc-text-h3"})
        playlist_count = count_playlist_in_one_person(driver, link_user)
        # more_info = soup.find("h3", {"class": "profileHeaderInfo__additional g-type-shrinkwrap-block theme-dark g-type-shrinkwrap-large-secondary sc-mt-1x"}).getText().Strip()

        #Add to one_data_user set
        one_data_user.append(id_user["content"].strip("soundcloud://users:"))
        one_data_user.append(name_user["content"])
        one_data_user.append(url_user["content"])
        one_data_user.append(playlist_count)
        # one_data_user.append(more_info)
        one_data_user.append(info_start[0].getText())   #followers
        one_data_user.append(info_start[1].getText())   #following
        one_data_user.append(info_start[2].getText())   #Track count
        one_data_user.append(likeTrack_count[1].getText().strip(' likes'))
        #Add one data user to set data 
        set_data_user.append(one_data_user)
        
        printProgressBar(i, len(set_link), "'%'")
        
    driver.close()
    return set_data_user

### Tập thông tin của người dùng trong set_data_user
<p>Kiểm tra xem có đúng số lượng giống như tập <b>set_link</b></p>

In [11]:
set_data_user = get_all_user_data(driver, set_link)
len(set_data_user)

[========= ] 99%  '%'

1001

### Sau đó viết tập dữ liệu người dùng ra file userHTML.csv

In [12]:
#write set data user to file <'user.csv'>
df = pd.DataFrame(set_data_user,columns=['User ID', 
                                         'User Name', 
                                         'User Link', 
                                         'Playlist Count', 
                                         'Followers', 
                                         'Following', 
                                         'Track Count', 
                                         'Like Tracks'])
df.to_csv('userHTML.csv', index=False, sep='\t', encoding='utf-8')

## 2. Thông tin từ track
<p>Lấy những dữ liệu cần thiết về bằng sử dụng modules của BeautifulSoup</p>

<p>Ban đầu:   khởi tạo tập để lưu trữ thông tin tracks <b>set_track </b></p>

In [13]:
set_tracks = []

### Khởi tạo hàm getTracks()
<p>Thông tin lấy được từ trang web <b><i>tracks</i></b> của người dùng </p>
<ol>
<li> ID của người dùng </li>
<li> ID của tracks </li>
<li> Tên của tracks </li>
<li> Thời gian tracks được ban hành </li>
<li> Số lượng lượt nghe của track </li>    
<li> Số lượng lượt like của track </li>    
<li> Số lượng lượt repost của track </li>        
</ol>

In [14]:
def getTracks(set_user):
    driver = webdriver.Chrome()
    set_tracks = []
    for i, item in enumerate(set_user):
        temp = item + "/tracks"
        driver.get(temp)

        wait(driver, selector_name='infoStats__value sc-font-light')

        scroll(driver)

        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        user_id = soup.find('meta', {'property': 'twitter:app:url:googleplay'})['content'].split(':')[-1]
        link_tracks = soup.find_all('a', {'class': 'sc-link-primary soundTitle__title sc-link-dark sc-text-h4'})
        link_tracks = link_tracks[:-3]
        info_start = soup.find_all("div", {"class": "infoStats__value sc-font-light"})
        track_count = info_start[2].getText()
        if track_count != '0':
            for link in link_tracks:
                string_temp = 'https://soundcloud.com'
                string_temp = string_temp + link['href']
                driver.get(string_temp)
                wait(driver, selector_name='sc-ministats-item')
                soup = BeautifulSoup(driver.page_source, "html.parser")
                id_track = soup.find('meta', {'property': 'twitter:app:url:googleplay'})['content'].split(':')[-1]
                name_track = soup.find('h1', {'class': 'soundTitle__title sc-font g-type-shrinkwrap-inline g-type-shrinkwrap-large-primary theme-dark'}).getText().strip()
                time_release_track = soup.find('time', {'class': 'relativeTime'})['title']
                info_track = soup.find_all('span', {'aria-hidden': 'true'})
                plays = info_track[1].getText()
                likes = info_track[2].getText()
                reposts = info_track[3].getText()

                tup = [user_id, id_track, name_track, time_release_track, plays, likes, reposts]
                set_tracks.append(tup)
        else:
            set_tracks.append([user_id, "None", "None", "None", "None", "None", "None"])
        
        printProgressBar(i, len(set_user), "'%'")
    driver.close()
    return set_tracks

### Tập thông tin của track trong set_tracks
<p> Kiểm tra <b>số lượng tracks</b> lấy được</p>

In [15]:
set_tracks = getTracks(set_link)
len(set_tracks)

[========= ] 94%  '%'

76

### Sau đó viết tập dữ liệu tracks ra file tracksHTML.csv

In [16]:
df = pd.DataFrame(set_tracks, columns=['User ID',
                                      'Tracks ID',
                                      'Tracks Name',
                                      'Time_Release_Track',
                                      'Plays',
                                      'Likes',
                                      'Repost'])
df.to_csv('tracksHTML.csv', index=False, sep='\t', encoding='utf-8')

## 3. Thông tin từ Playlist
<p>Lấy những dữ liệu cần thiết về bằng sử dụng modules của BeautifulSoup</p>

<p>Ban đầu:   khởi tạo tập để lưu trữ thông tin <b>các playlist</b> của người dùng <b>set_playlists </b></p>

In [18]:
set_playlists = []

<h3>Thông tin lấy được từ trang web <b><i>playlist và thông tin các tracks trong playlist</i></b> của người dùng </h3>
<ol>
<li> ID của người dùng </li>
<li> Tên của người dùng </li>
<li> Tên của Playlist </li>
<li> Số lượng tracks có trong playlist </li>
<li> Thời gian khởi tạo của playlist </li>    
<li> Số lượng lượt like của playlist </li>    
<li> Số lượng lượt repost của track </li>    
<li> Danh sách các tracks có trong play list được lưu dưới dạng: <b>ID_track_01, ID_track_02,... </b> </li>        
</ol>

### Khởi tạo hàm getTracksFromPlaylists(playlist_url, optional)
<h4> Trong đó có 2 tham số </h4><ol>
    <li><b>playlist_url</b> bao gồm các đường link đẫn đến trang của playlist đó</li>
    <li><b>optional</b> <ol>
        <li>Nếu optional == 0 thì chỉ lấy <b>tên của các track</b></li> 
        <li>Nếu optional == 1 thì driver truy cập vào từng track để lấy <b>id track</b></li>
</ol>            
</ol>    
<h4>Kết quả trả về là các dữ liệu cần trong tập playlists</h4>

In [55]:
def getTracksFromPlaylists(playlist_url, optional):
    driver = webdriver.Chrome()
    driver.get(playlist_url)
    wait(driver, selector_name='trackItem__trackTitle')
    scroll(driver)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    user_name = soup.find("a", {"class": "userBadge__usernameLink"}).getText().strip()
    playlist_title = soup.find("h1", {"class": "soundTitle__title"}).getText().strip()
    num_of_track = soup.find("div", {"class": "genericTrackCount__title"}).getText()
    create_date = soup.find("time", {"class": "relativeTime"})["title"]
    datas = soup.find_all("span", {"aria-hidden": "true"})

    likes = datas[1].getText()
    reports = datas[2].getText()

    track_list = []
    track_str = []

    if optional == 0:
        track_items = soup.find_all("a", {"class": "trackItem__trackTitle sc-link-dark sc-link-primary sc-font-light"})
        for track_item in track_items:
            track_name = 'https://soundcloud.com' + track_item['href']
            track_list.append(track_name.strip().replace("\n", " "))
    elif optional == 1:
        track_items = soup.find_all("a", {"class": "trackItem__trackTitle"})
        track_urls = []

        for track_item in track_items:
            track_urls.append('https://soundcloud.com' + track_item['href'])

        for url in track_urls:
            driver.get(url)
            wait(driver, selector_name='soundTitle__usernameTitleContainer')
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")

            meta = soup.find("meta", property="twitter:app:url:iphone")
            track_list.append(meta["content"].split(':')[-1])

    track_str = ','.join(track_list)
    if track_str == '':
        track_str = 'None'
    
    driver.close()
    return [user_name, playlist_title, num_of_track, create_date, likes, reports, track_str]

### Khởi tạo hàm getPlaylistURLs()
<p>Từ <b>user_playlist_url</b> ở đây tức là đường link đến các playlists của người dùng</p> 
    <h5>Kết quả trả về <b>tập các đường link</b> đến toàn bộ playlist của người dùng đó </h5> 

In [56]:
def getPlaylistURLs(user_playlist_url):
    driver = webdriver.Chrome()
    driver.get(user_playlist_url)
    scroll(driver)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    contents = soup.find_all("a", {"class": "sound__coverArt"})
    urls = []

    for content in contents:
        urls.append('https://soundcloud.com' + content['href'])
    
    driver.close()
    return urls

### Khởi tạo hàm getPlaylists()
<h4> Trong đó có bao gồm 2 hàm trên </h4><ol>
    <li><b>getPlaylistURLs</b>: để lấy được các đường link của playlist</li>
    <li><b>getTracksFromPlaylists</b>: để lấy được thông tin các track có trong playlist <ol>
        <li>Nếu optional == 0 thì chỉ lấy <b>tên của các track</b></li> 
        <li>Nếu optional == 1 thì driver truy cập vào từng track để lấy <b>id track</b></li>
</ol>            
</ol>    
<h4>Kết quả trả về là các dữ liệu cần trong tập playlists_lst</h4>

In [60]:
def getPlaylists(user_playlist_urls, optional):
    playlist_lst = []
    for i, user_url in enumerate(user_playlist_urls):
        driver = webdriver.Chrome()
        driver.get(user_url)
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        user_id = soup.find('meta', {'property': 'twitter:app:url:googleplay'})['content'].split(':')[-1]
        driver.close()
        playlist_urls = getPlaylistURLs(user_url + '/sets')
        
        for j, playlist_url in enumerate(playlist_urls):
            playlist_lst.append([user_id] + getTracksFromPlaylists(playlist_url, optional))
            printProgressBar(j, len(playlist_urls), '')
        
        printProgressBar(i, len(user_playlist_urls), "'%'")
    return playlist_lst

### Tập thông tin của playlists trong set_playlists
<p> Kiểm tra <b>số lượng playlist</b> lấy được</p>

In [61]:
set_playlists = getPlaylists(set_link, optional=0)
len(set_playlists)

[========= ] 94%  '%'

442

### Sau đó viết tập dữ liệu playlist ra file tracksHTML.csv

In [63]:
df = pd.DataFrame(set_playlists, columns=['User ID',
                                          'User name',
                                          'Playlist name',
                                          'Track count',
                                          'Create at',
                                          'Likes count',
                                          'Reposts count',
                                          'Track list'
                                         ])
df.to_csv('playlistHTML.csv', index=False, sep='\t', encoding='utf-8')

# Đọc dữ liệu/ file *.csv để kiểm tra 

In [12]:
df = pd.read_csv (r'userHTML.csv', sep='delimiter', header=None)
df.head()

C:\Users\tt\AppData\Local\Temp/ipykernel_1660/878794915.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv (r'userHTML.csv', sep='delimiter', header=None)


,0
0,User ID\tUser Name\tUser Link\tPlaylist Count\...
1,133467077\tWestcoast Radio (EDM)\thttps://soun...
2,214279944\tIlyana Azman\thttps://soundcloud.co...
3,79816260\tAustin24heck\thttps://soundcloud.com...
4,11164017\tUlana\thttps://soundcloud.com/itsula...


### Tiền xử lí file CSV

In [14]:
set_user = [i.split('\t')[2] for i in df[0]]

### Xuất giá trị đầu sau khi chỉnh sửa

In [17]:
set_user = set_user[0]
set_user

'https://soundcloud.com/austin24heck'